In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import geopy.distance
import unicodedata
import requests
from difflib import SequenceMatcher

In [2]:

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


def getDistance(lat1, lon1, lat2, lon2):  # get the distance between two points
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)

    return geopy.distance.geodesic(coord1, coord2).km


def get_local(address):
    try:
        url = f'https://nominatim.openstreetmap.org/search?q={address}&format=json&limit=1'
        location = requests.get(url=url)
        location = location.json()
        app = Nominatim(user_agent="test")
        location = app.geocode(address).raw  

        return(location['lat'], location['lon'])

    except:
        return None


def get_area(address):
    try:
        app = Nominatim(user_agent="test")
        location = app.geocode(address).raw
        lat1 = location['boundingbox'][0]
        lat2 = location['boundingbox'][1]
        lon1 = location['boundingbox'][2]
        lon2 = location['boundingbox'][3]

        side1 = getDistance(lat1, lon1, lat1, lon2)
        side2 = getDistance(lat1, lon2, lat2, lon2)

        area = side1*side2

        return area
    except:
        return None


def check_city(lat, lon):
    lat = str(lat)
    lon = str(lon)
    app2 = Nominatim(user_agent="test")
    location = app2.reverse(lat+","+lon)
    address = location.raw['address']
    city = address.get('city', '')

    if (city == 'Recife'):
        return True
    else:
        return False


def check_suburb(lat, lon, input_suburb, input_suburb2):

    lat = str(lat)
    lon = str(lon)
    app2 = Nominatim(user_agent="test")
    location = app2.reverse(lat+","+lon)
    address = location.raw['address']
    city = address.get('city', '')
    suburb = address.get('suburb', '')

    # filters the string suburb
    suburb = ''.join((c for c in unicodedata.normalize(
        'NFD', suburb) if unicodedata.category(c) != 'Mn'))
    suburb = suburb.lower()
    suburb = suburb.replace(" ", "")

    # filters the string input_suburb
    input_suburb = str(input_suburb)
    input_suburb = ''.join((c for c in unicodedata.normalize(
        'NFD', input_suburb) if unicodedata.category(c) != 'Mn'))
    input_suburb = input_suburb.lower()
    input_suburb = input_suburb.replace(" ", "")
    similarity = similar(suburb, input_suburb)

    input_suburb2 = str(input_suburb2)
    input_suburb2 = ''.join((c for c in unicodedata.normalize(
        'NFD', input_suburb2) if unicodedata.category(c) != 'Mn'))
    input_suburb2 = input_suburb2.lower()
    input_suburb2 = input_suburb2.replace(" ", "")

    similarity2 = similar(suburb, input_suburb2)

    if (similarity > 0.6 and city == 'Recife'):
        return True
    else:
        if (similarity2 > 0.6 and city == 'Recife'):
            return True
        else:
            print(input_suburb, input_suburb2, suburb, similarity, similarity2)
            return False

while (True):
    df = pd.read_csv("merged.csv")
    actual_df = pd.read_csv("../rain_elevation/location.csv")

    df['latitude'] = ''
    df['longitude'] = ''
    df['cond'] = ''


    df['processo_numero'] = df['processo_numero'].astype(str)
    actual_df['processo_numero'] = actual_df['processo_numero'].astype(str)
    df['cond']= df['processo_numero'].isin(actual_df['processo_numero'])

    df = df[df.cond != True]


    df = df[df.confirmado != 2]

    print(df.info())

    total = len(df)
    print(total)

    df = df.sample(frac=1)
    df = df[:1]

    achou = 0

    try:

        for counter, bairro in enumerate(df['solicitacao_bairro']):

            print("!!!", counter, "!!!")
            print("---", achou*100/(counter+1),"%---")
            bairro = str(bairro)
            bairro = bairro.replace("CGO. ", "CÓRREGO ")
            bairro = bairro.replace("JD. ", "CÓRREGO ")
            bairro = bairro.replace("AV ", "AVENIDA ")
            bairro_recife = bairro + ' Recife '

            rua = str(df.iat[counter, df.columns.get_loc('solicitacao_endereco')])
            rua = rua.upper()
            rua = rua.replace("CGO. ", "CÓRREGO ")
            rua = rua.replace("JD. ", "CÓRREGO ")
            rua = rua.replace("TRV ", "TRAVESSA ")
            rua = rua.replace("EST ", "ESTRADA")
            rua = rua.replace("AV ", "Avenida ")
            rua = rua.replace(",", " ")
            rua = rua.replace(".", " ")
            rua = rua.replace("º", " ")
            rua = rua.replace("°", " ")
            rua = rua.replace(" N ", " ")

            rua_recife = rua + ' Recife '

            localidade = str(
                df.iat[counter, df.columns.get_loc('solicitacao_localidade')])
            localidade = localidade.replace("CGO. ", "CÓRREGO ")
            localidade = localidade.replace("JD. ", "CÓRREGO ")
            localidade = localidade.replace("AV ", "Avenida ")

            localidade_recife = localidade + ' Recife '

            first_try = rua_recife + bairro

            if (get_local(first_try) != None):
                lat_and_lon = get_local(first_try)
                lat = lat_and_lon[0]
                lon = lat_and_lon[1]
                print("Achou lat e lon1")

                if (check_suburb(lat, lon, bairro, localidade) == True):
                    df.iat[counter, df.columns.get_loc('latitude')] = lat
                    df.iat[counter, df.columns.get_loc('longitude')] = lon
                    print("Achou cidade com bairro", first_try)
                    achou += 1

                else:
                    if (get_local(localidade_recife) != None):
                        lat_and_lon2 = get_local(localidade_recife)
                    else:
                        lat_and_lon2 = get_local(bairro)

                    try:
                        lat2 = lat_and_lon2[0]
                        lon2 = lat_and_lon2[1]
                    except:
                        continue
                    dist = getDistance(lat, lon, lat2, lon2)
                    if (dist < 1.5):
                        print("Achou cidade com distancia", dist)
                        df.iat[counter, df.columns.get_loc('latitude')] = lat
                        df.iat[counter, df.columns.get_loc('longitude')] = lon
                        achou += 1
                    else:
                        print("Lugar errado", bairro)

            else:

                second_try = rua_recife + localidade

                if (get_local(second_try) != None):
                    lat_and_lon = get_local(second_try)
                    lat = lat_and_lon[0]
                    lon = lat_and_lon[1]
                    print("Achou lat e lon2")

                    if (check_suburb(lat, lon, bairro, localidade) == True):
                        df.iat[counter, df.columns.get_loc('latitude')] = lat
                        df.iat[counter, df.columns.get_loc('longitude')] = lon
                        print("Achou cidade")
                        achou += 1
                    else:
                        if (get_local(localidade_recife) != None):
                            lat_and_lon2 = get_local(localidade_recife)
                        else:
                            lat_and_lon2 = get_local(bairro)

                        try:
                            lat2 = lat_and_lon2[0]
                            lon2 = lat_and_lon2[1]
                        except:
                            continue
                        dist = getDistance(lat, lon, lat2, lon2)
                        if (dist < 1.5):
                            print("Achou cidade com distancia", dist)
                            df.iat[counter, df.columns.get_loc('latitude')] = lat
                            df.iat[counter, df.columns.get_loc('longitude')] = lon
                            achou += 1
                        else:
                            print("Lugar errado", bairro)
                else:

                    third_try = rua_recife

                    if (get_local(third_try) != None):
                        lat_and_lon = get_local(third_try)
                        lat = lat_and_lon[0]
                        lon = lat_and_lon[1]
                        print("Achou lat e lon3")

                        if (check_suburb(lat, lon, bairro, localidade) == True):
                            df.iat[counter, df.columns.get_loc('latitude')] = lat
                            df.iat[counter, df.columns.get_loc('longitude')] = lon
                            print("Achou cidade")
                            achou += 1
                        else:
                            if (get_local(localidade_recife) != None):
                                lat_and_lon2 = get_local(localidade_recife)
                            else:
                                lat_and_lon2 = get_local(bairro)

                            try:
                                lat2 = lat_and_lon2[0]
                                lon2 = lat_and_lon2[1]
                            except:
                                continue
                            dist = getDistance(lat, lon, lat2, lon2)

                            if (dist < 1.5):
                                print("Achou cidade com distancia", dist)
                                df.iat[counter, df.columns.get_loc(
                                    'latitude')] = lat
                                df.iat[counter, df.columns.get_loc(
                                    'longitude')] = lon
                                achou += 1
                            else:
                                print("Lugar errado", bairro)

                    else:

                        fourth_try = localidade_recife
                        if (get_area(fourth_try) != None):
                            area = get_area(fourth_try)

                            if (area < 7 and area != None):
                                lat_and_lon = get_local(fourth_try)
                                lat = lat_and_lon[0]
                                lon = lat_and_lon[1]

                                df.iat[counter, df.columns.get_loc(
                                    'latitude')] = lat
                                df.iat[counter, df.columns.get_loc(
                                    'longitude')] = lon
                                achou += 1

                                print("Achou lat e lon na localidade 4")
                            else:
                                print(bairro, localidade, rua)
                                print("Localidade > 7km2", area)

                        else:
                            print(bairro, localidade, rua)
                            print("Não achou no geral")

        df = df[df.latitude != '']
        df['em_recife'] = ''

        for counter, lat_lon in enumerate(df["solicitacao_bairro"]):
            lat = df.iat[counter, df.columns.get_loc('latitude')]
            lon = df.iat[counter, df.columns.get_loc('longitude')]
            if (lat != ''):
                lat = float(lat)
                lon = float(lon)

                if (check_city(lat, lon) == False):
                    df.iat[counter, df.columns.get_loc('em_recife')] = 0
                else:
                    print("Esta em Recife")
                    df.iat[counter, df.columns.get_loc('em_recife')] = 1

        df = df[df.em_recife != 0]
        df = pd.concat([df, actual_df])

        print(f"----", {achou/total*100}, "%-----")
        df.to_csv(r'C:\Users\parae\Documents\barreiras\barreiras\processing\rain_elevation\location.csv',
                index=False, header=True)

    except:

        df = df[df.latitude != '']
        df['em_recife'] = ''

        for counter, lat_lon in enumerate(df["solicitacao_bairro"]):
            lat = df.iat[counter, df.columns.get_loc('latitude')]
            lon = df.iat[counter, df.columns.get_loc('longitude')]
            if (lat != ''):
                lat = float(lat)
                lon = float(lon)

                if (check_city(lat, lon) == False):
                    df.iat[counter, df.columns.get_loc('em_recife')] = 0
                else:
                    print("Esta em Recife")
                    df.iat[counter, df.columns.get_loc('em_recife')] = 1

        df = df[df.em_recife != 0]
        df = pd.concat([df, actual_df])
        print(f"----", {achou/total*100}, "%-----")
        df.to_csv(r'C:\Users\parae\Documents\barreiras\barreiras\processing\rain_elevation\location.csv',
                index=False, header=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21713 entries, 31 to 60520
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   processo_numero         21713 non-null  object
 1   solicitacao_data        21713 non-null  object
 2   solicitacao_hora        21713 non-null  object
 3   solicitacao_descricao   21713 non-null  object
 4   solicitacao_bairro      21713 non-null  object
 5   solicitacao_localidade  21713 non-null  object
 6   solicitacao_endereco    21713 non-null  object
 7   processo_ocorrencia     21713 non-null  object
 8   confirmado              21713 non-null  int64 
 9   latitude                21713 non-null  object
 10  longitude               21713 non-null  object
 11  cond                    21713 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 2.0+ MB
None
21713
!!! 0 !!!
--- 0.0 %---
Achou lat e lon na localidade 4
Esta em Recife


C:\Users\parae\AppData\Local\Temp\ipykernel_11536\778069468.py:312: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df = pd.concat([df, actual_df])


---- {0.004605535854096624} %-----
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21712 entries, 31 to 60520
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   processo_numero         21712 non-null  object
 1   solicitacao_data        21712 non-null  object
 2   solicitacao_hora        21712 non-null  object
 3   solicitacao_descricao   21712 non-null  object
 4   solicitacao_bairro      21712 non-null  object
 5   solicitacao_localidade  21712 non-null  object
 6   solicitacao_endereco    21712 non-null  object
 7   processo_ocorrencia     21712 non-null  object
 8   confirmado              21712 non-null  int64 
 9   latitude                21712 non-null  object
 10  longitude               21712 non-null  object
 11  cond                    21712 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 2.0+ MB
None
21712
!!! 0 !!!
--- 0.0 %---
Achou lat e lon1
Achou cidade com

C:\Users\parae\AppData\Local\Temp\ipykernel_11536\778069468.py:312: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df = pd.concat([df, actual_df])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21711 entries, 31 to 60520
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   processo_numero         21711 non-null  object
 1   solicitacao_data        21711 non-null  object
 2   solicitacao_hora        21711 non-null  object
 3   solicitacao_descricao   21711 non-null  object
 4   solicitacao_bairro      21711 non-null  object
 5   solicitacao_localidade  21711 non-null  object
 6   solicitacao_endereco    21711 non-null  object
 7   processo_ocorrencia     21711 non-null  object
 8   confirmado              21711 non-null  int64 
 9   latitude                21711 non-null  object
 10  longitude               21711 non-null  object
 11  cond                    21711 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 2.0+ MB
None
21711
!!! 0 !!!
--- 0.0 %---
Achou lat e lon1
Achou cidade com bairro RUA IBIRAJUBA Recife IBURA


...

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20299 entries, 31 to 60520
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   processo_numero         20299 non-null  object
 1   solicitacao_data        20299 non-null  object
 2   solicitacao_hora        20299 non-null  object
 3   solicitacao_descricao   20299 non-null  object
 4   solicitacao_bairro      20299 non-null  object
 5   solicitacao_localidade  20299 non-null  object
 6   solicitacao_endereco    20299 non-null  object
 7   processo_ocorrencia     20299 non-null  object
 8   confirmado              20299 non-null  int64 
 9   latitude                20299 non-null  object
 10  longitude               20299 non-null  object
 11  cond                    20299 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 1.9+ MB
None
20299
!!! 0 !!!
--- 0.0 %---
Achou lat e lon3
novadescoberta novadescoberta vascodagama 0.48 0.48

C:\Users\parae\AppData\Local\Temp\ipykernel_11536\778069468.py:312: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df = pd.concat([df, actual_df])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20299 entries, 31 to 60520
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   processo_numero         20299 non-null  object
 1   solicitacao_data        20299 non-null  object
 2   solicitacao_hora        20299 non-null  object
 3   solicitacao_descricao   20299 non-null  object
 4   solicitacao_bairro      20299 non-null  object
 5   solicitacao_localidade  20299 non-null  object
 6   solicitacao_endereco    20299 non-null  object
 7   processo_ocorrencia     20299 non-null  object
 8   confirmado              20299 non-null  int64 
 9   latitude                20299 non-null  object
 10  longitude               20299 non-null  object
 11  cond                    20299 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 1.9+ MB
None
20299
!!! 0 !!!
--- 0.0 %---
Achou lat e lon3
ibura corregomonteverde cohab 0.2 0.181818181818181

C:\Users\parae\AppData\Local\Temp\ipykernel_11536\778069468.py:312: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df = pd.concat([df, actual_df])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 20299 entries, 31 to 60520
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   processo_numero         20299 non-null  object
 1   solicitacao_data        20299 non-null  object
 2   solicitacao_hora        20299 non-null  object
 3   solicitacao_descricao   20299 non-null  object
 4   solicitacao_bairro      20299 non-null  object
 5   solicitacao_localidade  20299 non-null  object
 6   solicitacao_endereco    20299 non-null  object
 7   processo_ocorrencia     20299 non-null  object
 8   confirmado              20299 non-null  int64 
 9   latitude                20299 non-null  object
 10  longitude               20299 non-null  object
 11  cond                    20299 non-null  bool  
dtypes: bool(1), int64(1), object(10)
memory usage: 1.9+ MB
None
20299
!!! 0 !!!
--- 0.0 %---
